In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

c:\Users\victo\anaconda3\envs\montecarlo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def successor(X, s, r, delta, m, T):
    h = T/m
    z = np.random.normal()
    return X * np.exp((r - delta - (0.5 * s**2)) * h + s * np.sqrt(h) * z)

def black_scholes(X0, s, r, delta, m, T):
    X = [X0]
    for i in range(m):
        X.append(successor(X[-1], s, r, delta, m, T))
    return X

def put_func(a, K, r, t):
    """
    Payoff function for a Bermudan put option
    """
    return np.exp(-r*t) * np.maximum(K - a, 0)

def call_func(a, K, r, t):
    """
    Payoff function for a Bermudan call option
    """
    return np.exp(-r*t) * np.maximum(a - K, 0)

def high_estimator(payoff_func, current, nextt, K, r, t, h):
    discount_factor = np.exp(-r*h)
    return max(payoff_func(current, K, r, t), np.exp(-r*t)*np.mean(nextt))

# def low_estimator(payoff_func, current, nextt, K, r, t):
#     b = len(nextt)
#     v_ = np.zeros(b)
#     for k in range(b):
#         new = np.concatenate([nextt[:k], nextt[k+1:]]) # remove the k-th element
#         if np.mean(new) <= payoff_func(current, K, r, t):
#             v_[k] = payoff_func(current, K, r, t)
#         else:
#             v_[k] = nextt[k]
#     return np.mean(v_)


# Faster low estimator
def low_estimator(payoff_func, current, nextt, K, r, t, h):
    b = len(nextt)
    v_ = np.zeros(b)
    discount_factor = np.exp(-r*h)
    mean = discount_factor * np.mean(nextt[1:])

    if mean <= payoff_func(current, K, r, t):
        v_[0] = payoff_func(current, K, r, t)
    else:
        v_[0] = nextt[0]

    for k in range(1, b):
        mean += nextt[k-1]/(b-1) * discount_factor
        mean -= nextt[k]/(b-1) * discount_factor
        if mean <= payoff_func(current, K, r, t):
            v_[k] = payoff_func(current, K, r, t)
        else:
            v_[k] = nextt[k]
    return np.mean(v_)

In [15]:
def rtree(X0, m, b, s, r, delta, K, T, payoff_func):
    h = T/m

    w = np.zeros(m+1).astype(int)     # to know at which branch we are
    # Values of the estimators and asset
    X = np.zeros((b, m+1))
    v = np.zeros((b, m+1))   
    V = np.zeros((b, m+1))   

    # Root nodes
    X[0, 0] = X0

    # Initialize the first branches
    for j in range(1, m+1):
        X[0, j] = successor(X[0, j-1], s, r, delta, m, T)
    
    j = m

    while j >= 1:
        if j == m and w[j] < b-1:
            v[w[j], j] = payoff_func(X[w[j], j], K, r, j*h)
            V[w[j], j] = payoff_func(X[w[j], j], K, r, j*h)
            X[w[j]+1, j] = successor(X[w[j-1], j-1], s, r, delta, m, T)
            w[j] += 1
        
        elif j == m and w[j] == b-1:
            v[w[j], j] = payoff_func(X[w[j], j], K, r, j*h)
            V[w[j], j] = payoff_func(X[w[j], j], K, r, j*h)
            w[j] = 0
            j -= 1
        
        elif j < m and w[j] < b-1:
            v[w[j], j] = low_estimator(payoff_func, X[w[j], j], v[:, j+1], K, r, j*h, h)
            V[w[j], j] = high_estimator(payoff_func, X[w[j], j], V[:, j+1], K, r, j*h, h)
            if j > 0:
                w[j] += 1
                X[w[j], j] = successor(X[w[j-1], j-1], s, r, delta, m, T)
                for i in range(j+1, m+1):
                    X[0, i] = successor(X[w[j], i-1], s, r, delta, m, T)
                j = m
            else:
                break
        
        elif j < m and w[j] == b-1:
            v[w[j], j] = low_estimator(payoff_func, X[w[j], j], v[:, j+1], K, r, j*h, h)
            V[w[j], j] = high_estimator(payoff_func, X[w[j], j], V[:, j+1], K, r, j*h, h)
            w[j] = 0
            j -= 1

    return low_estimator(payoff_func, X0, v[:, 1], K, r, 0, h), high_estimator(payoff_func, X0, V[:, 1], K, r, 0, h)

In [16]:
def monte_carlo(n, X0, m, b, s, r, delta, K, T, payoff_func):
    eff_low = []
    eff_high = []
    for _ in tqdm(range(n)):
        eff_low.append(rtree(X0, m, b, s, r, delta, K, T, payoff_func)[0])
        eff_high.append(rtree(X0, m, b, s, r, delta, K, T, payoff_func)[1])
    return np.array(eff_low), np.array(eff_high)

In [19]:
n = 100
X0 = 100
m = 3
b = 50
s = 0.2
r = 0.1
delta = 0.05
K = 100
T = 1
eff_low, eff_high = monte_carlo(n, X0, m, b, s, r, delta, K, T, put_func)

 39%|███▉      | 39/100 [08:23<13:08, 12.92s/it]


KeyboardInterrupt: 

In [217]:
parameters = [n, X0, m, b, s, r, delta, K, T, 'put']
np.save('parametersb50.npy', parameters)
np.save('runb50.npy', [eff_low, eff_high])

In [6]:
parameters = np.load('parametersb50.npy')
eff_low, eff_high = np.load('runb50.npy')

In [18]:
mean_v = np.mean(eff_low)
mean_V = np.mean(eff_high)

z = 1.96

std_v = 1/(n-1) * np.sum((eff_low - mean_v)**2)
std_V = 1/(n-1) * np.sum((eff_high - mean_V)**2)

std_errlow = z * std_v / np.sqrt(n)
std_errhigh = z * std_V / np.sqrt(n)
print('90% confidence interval: [', mean_v - std_errlow, mean_V + std_errhigh,']')

print('Low estimator:', mean_v)
print('High estimator:', mean_V)
print('Std error for the low estimator:', std_errlow)
print('Std for the high estimator:', std_errhigh)
print('Point estimate:', 0.5 * (mean_v + mean_V))

points = 0.5 * (eff_high + eff_low)

90% confidence interval: [ 5.229328786428353 6.033108428914634 ]
Low estimator: 5.474537898558179
High estimator: 5.717184851644716
Std error for the low estimator: 0.24520911212982574
Std for the high estimator: 0.31592357726991815
Point estimate: 5.595861375101448


With variance reduction techniques